In [ ]:
#Genero ruta

In [ ]:
import geopandas as gpd
import zipfile
import os
import geopandas as gpd
import shapely.geometry
import shapely.ops
from shapely.geometry import LineString, Point
import pyproj
import random
import requests
from PIL import Image
from io import BytesIO
#from moviepy.editor import ImageSequenceClip
#from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip


In [ ]:
api_key =

Helena

In [ ]:
from shapely.ops import linemerge

# Leer el archivo GeoJSON
geojson_path1 = 'ruta1.geojson'
gdf_1 = gpd.read_file(geojson_path1)

# Asegurarse de que la geometría sea una sola línea (LineString)
if gdf_1.geometry.type[0] == 'MultiLineString':
    line = linemerge(gdf_1.geometry[0])
elif gdf_1.geometry.type[0] == 'LineString':
    line = gdf_1.geometry[0]
else:
    raise ValueError('La geometría del archivo no es una LineString ni una MultiLineString.')

# Proyectar a un CRS métrico (por ejemplo, UTM)
utm_crs = gdf_1.estimate_utm_crs()
line = gpd.GeoSeries([line], crs=gdf_1.crs).to_crs(utm_crs)[0]

# Calcular la longitud total de la línea en kilómetros
total_length_km = line.length / 1000
print(f"Longitud total de la línea: {total_length_km} km")

# Generar puntos cada 20 km
distance_interval = 20  # 20 km
num_points = int(total_length_km // distance_interval)

points = [line.interpolate(distance_interval * i * 1000) for i in range(num_points + 1)]

# Transformar los puntos de vuelta al CRS original (WGS84)
gdf_points = gpd.GeoSeries(points, crs=utm_crs).to_crs(epsg=4326)

# Crear la lista de pares de coordenadas (latitud, longitud)
coordinates_list1 = [(point.y, point.x) for point in gdf_points]

# Imprimir la lista de pares de coordenadas
for coord in coordinates_list1:
    print(coord)


In [ ]:
# Leer el archivo GeoJSON
geojson_path2 = 'ruta2.geojson'
gdf_2 = gpd.read_file(geojson_path2)

# Asegurarse de que la geometría sea una sola línea (LineString)
if gdf_2.geometry.type[0] == 'MultiLineString':
    line = shapely.ops.linemerge(gdf_2.geometry[0])
elif gdf_2.geometry.type[0] == 'LineString':
    line = gdf_2.geometry[0]
else:
    raise ValueError('La geometría del archivo no es una LineString ni una MultiLineString.')

# Proyectar a un CRS métrico (por ejemplo, UTM)
utm_crs = gdf_2.estimate_utm_crs()
gdf_2 = gdf_2.to_crs(utm_crs)
line = gdf_2.geometry[0]

# Calcular la longitud total de la línea en kilómetros
total_length_km = line.length / 1000

# Generar puntos cada 1 km
distance_interval = 20  # 20 km
num_points = int(total_length_km // distance_interval)

points = [line.interpolate(distance_interval * i * 1000) for i in range(num_points + 1)]

# Transformar los puntos de vuelta al CRS original (WGS84)
gdf_points = gpd.GeoSeries(points, crs=utm_crs).to_crs(epsg=4326)

# Crear la lista de pares de coordenadas (latitud, longitud)
coordinates_list2 = [(point.y, point.x) for point in gdf_points]

# Imprimir la lista de pares de coordenadas
for coord in coordinates_list2:
    print(coord)

In [ ]:
coordinates_list = coordinates_list1 + coordinates_list2

In [ ]:
coordinates_list

In [ ]:
import folium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
from PIL import Image

# Configuración de Selenium
def setup_driver(output_dir):
    options = Options()
    options.add_argument("--headless")  # Ejecución sin interfaz gráfica
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    prefs = {"download.default_directory": output_dir}
    options.add_experimental_option("prefs", prefs)
    service = Service()  # Usa el path correcto de tu driver si es necesario
    return webdriver.Chrome(service=service, options=options)

# Función para capturar mapa
def capture_map(lat, lon, zoom=15, output_dir="maps", filename=None, size=(640, 640)):
    # Crear mapa con folium
    m = folium.Map(location=[lat, lon], zoom_start=zoom, width=size[0], height=size[1])
    folium.Marker([lat, lon]).add_to(m)
    
    # Guardar mapa como archivo HTML temporal
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    html_path = os.path.join(output_dir, "temp_map.html")
    m.save(html_path)
    
    # Configurar Selenium
    driver = setup_driver(output_dir)
    driver.get(f"file://{os.path.abspath(html_path)}")
    driver.set_window_size(size[0], size[1])
    
    # Capturar pantalla
    screenshot_path = os.path.join(output_dir, filename or f"map_{lat}_{lon}.png")
    driver.save_screenshot(screenshot_path)
    driver.quit()
    
    # Recortar la imagen al tamaño deseado
    image = Image.open(screenshot_path)
    cropped_image = image.crop((0, 0, size[0], size[1]))
    cropped_image.save(screenshot_path)

    print(f"Mapa guardado: {screenshot_path}")
    return screenshot_path

# Lista de coordenadas (ejemplo)
coordinates = coordinates_list
# Capturar mapas
for i, (lat, lon) in enumerate(coordinates):
    capture_map(lat, lon, filename=f"map_{i}.png")


In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
import folium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
from PIL import Image
import shapely.ops

# Crear GeoDataFrame a partir de la lista de coordenadas
points = [Point(lon, lat) for lat, lon in coordinates_list]
gdf_points = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")  # Establecer CRS WGS 84

# Crear la línea combinada de las coordenadas
combined_line = LineString(points)

# Proyectar a un CRS métrico (por ejemplo, UTM zona 36N)
utm_crs = "EPSG:32636"  # UTM zona 36N
combined_line_utm = gpd.GeoSeries([combined_line], crs="EPSG:4326").to_crs(utm_crs)[0]

# Calcular la longitud total de la línea en kilómetros
total_length_km = combined_line_utm.length / 1000
print(f"Longitud total de la línea combinada: {total_length_km} km")

# Generar puntos a intervalos regulares (20 km)
distance_interval = 20  # 20 km
num_points = int(total_length_km // distance_interval)
points_on_line = [combined_line_utm.interpolate(distance_interval * i * 1000) for i in range(num_points + 1)]

# Transformar los puntos de vuelta al CRS original (WGS84)
gdf_points = gpd.GeoSeries(points_on_line, crs=utm_crs).to_crs(epsg=4326)

# Crear la lista de pares de coordenadas (latitud, longitud)
coordinates_list = [(point.y, point.x) for point in gdf_points]

# Configuración de Selenium
def setup_driver(output_dir):
    options = Options()
    options.add_argument("--headless")  # Ejecución sin interfaz gráfica
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    prefs = {"download.default_directory": output_dir}
    options.add_experimental_option("prefs", prefs)
    service = Service()  # Asegúrate de usar el path correcto si es necesario
    return webdriver.Chrome(service=service, options=options)

# Función para capturar un mapa con folium
def capture_map_with_line_and_point(map_center, line_coords, point_coord, zoom=10, output_dir="maps", filename=None, size=(640, 640)):
    # Crear el mapa centrado
    m = folium.Map(location=map_center, zoom_start=zoom, width=size[0], height=size[1])
    
    # Agregar la línea del recorrido
    folium.PolyLine(line_coords, color="blue", weight=4).add_to(m)
    
    # Agregar el punto de avance
    folium.Marker(location=point_coord, icon=folium.Icon(color="red")).add_to(m)
    
    # Guardar el mapa como archivo HTML temporal
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    html_path = os.path.join(output_dir, "temp_map.html")
    m.save(html_path)
    
    # Configurar Selenium
    driver = setup_driver(output_dir)
    driver.get(f"file://{os.path.abspath(html_path)}")
    driver.set_window_size(size[0], size[1])
    
    # Capturar pantalla
    screenshot_path = os.path.join(output_dir, filename or f"map_{point_coord[0]}_{point_coord[1]}.png")
    driver.save_screenshot(screenshot_path)
    driver.quit()
    
    # Recortar la imagen al tamaño deseado
    image = Image.open(screenshot_path)
    cropped_image = image.crop((0, 0, size[0], size[1]))
    cropped_image.save(screenshot_path)
    print(f"Mapa guardado: {screenshot_path}")
    return screenshot_path

# Generar mapas para los puntos de avance en el recorrido combinado
output_dir = "maps_combined"
for i, coord in enumerate(coordinates_list):
    capture_map_with_line_and_point(
        map_center=coord, 
        line_coords=coordinates_list, 
        point_coord=coord, 
        output_dir=output_dir,
        filename=f"map_combined_{i}.png"
    )



In [ ]:
import geopandas as gpd
import folium
from shapely.geometry import LineString, Point
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
from PIL import Image
import shapely.ops

# Lista de coordenadas (latitud, longitud)

# Crear GeoDataFrame a partir de la lista de coordenadas
points = [Point(lon, lat) for lat, lon in coordinates_list]
gdf_points = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")  # Establecer CRS WGS 84

# Crear la línea combinada de las coordenadas
combined_line = LineString(points)

# Calcular el bounding box del recorrido completo (de la línea)
minx, miny, maxx, maxy = combined_line.bounds

# Calcular el centro del recorrido (promedio de los límites)
center_lat = 34.81360204565566
center_lon = 32.74780672158749

# Configuración de Selenium
def setup_driver(output_dir):
    options = Options()
    options.add_argument("--headless")  # Ejecución sin interfaz gráfica
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    prefs = {"download.default_directory": output_dir}
    options.add_experimental_option("prefs", prefs)
    service = Service()  # Asegúrate de usar el path correcto si es necesario
    return webdriver.Chrome(service=service, options=options)

# Función para capturar un mapa con folium
def capture_map_with_line_and_point(map_center, line_coords, point_coord, zoom, output_dir="maps", filename=None, size=(640, 640)):
    # Crear el mapa centrado
    m = folium.Map(location=map_center, zoom_start=zoom, width=size[0], height=size[1])
    
    # Agregar la línea del recorrido
    folium.PolyLine(line_coords, color="blue", weight=4).add_to(m)
    
    # Agregar el marcador de un punto
    folium.Marker(location=point_coord, icon=folium.Icon(color="red")).add_to(m)
    
    # Guardar el mapa como archivo HTML temporal
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    html_path = os.path.join(output_dir, "temp_map.html")
    m.save(html_path)
    
    # Configurar Selenium
    driver = setup_driver(output_dir)
    driver.get(f"file://{os.path.abspath(html_path)}")
    driver.set_window_size(size[0], size[1])
    
    # Capturar pantalla
    screenshot_path = os.path.join(output_dir, filename or f"map_{point_coord[0]}_{point_coord[1]}.png")
    driver.save_screenshot(screenshot_path)
    driver.quit()
    
    # Recortar la imagen al tamaño deseado
    image = Image.open(screenshot_path)
    cropped_image = image.crop((0, 0, size[0], size[1]))
    cropped_image.save(screenshot_path)
    print(f"Mapa guardado: {screenshot_path}")
    return screenshot_path

# Calcular el nivel de zoom adecuado para que el recorrido entero sea visible
# Aproximación: si la distancia entre los extremos de la línea es pequeña, aumentamos el zoom
# Si la distancia es grande, disminuimos el zoom
distance = combined_line.length / 1000  # Distancia en kilómetros

# Definir el rango de zoom para que sea dinámico según la distancia

zoom_level = 5.25  # Zoom bajo para áreas grandes

# Crear el mapa base con el recorrido entero visible
output_dir = "maps_combined"
capture_map_with_line_and_point(
    map_center=(center_lat, center_lon),  # Centro basado en el bounding box
    line_coords=coordinates_list,  # Usamos la lista de coordenadas del recorrido
    point_coord=(center_lat, center_lon),  # Coordenada de centro
    zoom=zoom_level,  # Ajustamos el nivel de zoom
    output_dir=output_dir,
    filename="full_map.png"
)

# Generar mapas para cada punto con el marcador
for i, coord in enumerate(coordinates_list):
    capture_map_with_line_and_point(
        map_center=(center_lat, center_lon),  # Mantener el centro del recorrido fijo
        line_coords=coordinates_list,  # Recorrido completo
        point_coord=coord,  # Solo el marcador en este punto
        zoom=zoom_level,  # Mantener el zoom
        output_dir=output_dir,
        filename=f"map_with_marker_{i}.png"
    )


In [ ]:
len(coordinates_list2)

In [ ]:
# Definir la URL base para la API de Street View
base_url = "https://maps.googleapis.com/maps/api/streetview?size=3600x3600"

# API key
api_key = api_key

# Crear un directorio para guardar las imágenes
output_dir = 'streetview_images_2'
os.makedirs(output_dir, exist_ok=True)

# Generar y mostrar URLs con coordenadas y valores aleatorios de heading y pitch
for i, coord in enumerate(coordinates_list):
    lat, lon = coord
    fov = random.randint(30, 80)
    heading = random.randint(1, 359)
    pitch = random.randint(0, 40)
    url = f"{base_url}&location={lat},{lon}&fov=80&heading={heading}&pitch={pitch}&key={api_key}"
    print(url)

    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image_path = os.path.join(output_dir, f'image_{i+1}.jpg')
        image.save(image_path)
        print(f'Imagen guardada en {image_path}')
    else:
        print(f'Error al descargar la imagen de {url}')

print('Descarga de imágenes completada.')

In [ ]:
response = requests.get(url)
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    image_path = os.path.join(output_dir, f'image_{500}.jpg')
    image.save(image_path)
    print(f'Imagen guardada en {image_path}')
else:
    print(f'Error al descargar la imagen de {url}')

In [ ]:
# Crear un video con las imágenes
image_dir = 'streetview_images3'
fps = 0.5  # 3 segundos por imagen
image_files = sorted([os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.jpg')])
clip = ImageSequenceClip(image_files, fps=fps)

# Asegurarnos de que el directorio de salida exista
output_dir = os.path.dirname(image_files[0])  # Tomamos el directorio de la primera imagen
video_path = os.path.join(output_dir, 'streetview_video.mp4')

clip.write_videofile(video_path, codec='libx264',fps=fps)  # Asegurar que se pasa fps aquí también

print(f'Video guardado en {video_path}')

In [ ]:
from moviepy.config import change_settings

change_settings({"IMAGEMAGICK_BINARY": r"D:/Archivos de Programa/ImageMagick-7.1.1-Q16-HDRI/magick.exe"})

In [ ]:
from moviepy.editor import AudioFileClip

# Texto original dividido en subtítulos más pequeños
# Cargar el video original
video_clip = VideoFileClip('streetview_images3/streetview_video.mp4')
audio_clip = AudioFileClip('streetview_images3/Arooj Aftab, Vijay Iyer, Shahzad Ismaily - Sajni (Visualizer).mp3').set_duration(video_clip.duration)

subtitle_texts = [
    "Tuvo que haber un momento",
    "en el que se te aproximó la pregunta",
    "quizás con pasos más leves,", 
    "o más pesados",
    "cuando estuvo en tus manos",
    "si es que estuvo en tus manos",
    "si es que pudiste sostenerla",
    "¿Qué es una reliquia?",
    "Reliquiae: “lo que queda, los restos de la comida",
    "los sobrevivientes de una batalla, de un difunto”.",
    "La parte que se abandonó",
    "después de la pérdida o ruina del resto.",
    "Relinquere: “dejar atrás, desamparar,",
    "abandonar, rendirse”.",
    "¿Cómo se busca lo que se dejó atrás?",
    "¿Es más importante encontrarlo",
    "o llevarlo de vuelta?",
    "No pensé en tu vuelta.",
    "Sólo pensé en el viaje de ida.",
    "Dicen que te visitaron dos ángeles",
    "y te revelaron el lugar.",
    "Tres pedazos de madera y una placa.",
    "“Cualquier cosa que toma valor",
    "por su asociación con el pasado lejano”",
    '¿Qué es el pasado lejano?',
    "Tu recompensa,",
    "primera peregrina.",
    "Y aún así, dudaste.",
    "Tenés que haber dudado."
]
# Calcular la duración de cada subtítulo
total_duration = video_clip.duration
subtitle_duration = total_duration / len(subtitle_texts)*0.95

# Crear una lista de clips de texto con tiempos específicos
subtitle_clips = []
for i, text in enumerate(subtitle_texts):
    start_time = i * subtitle_duration
    # Ajustar la posición un poco más arriba y usar color rojo oscuro
    subtitle_clip = TextClip(text, fontsize=26, color='black', font='Arial').set_position(('center', video_clip.h - 80)).set_duration(subtitle_duration).set_start(start_time)
    subtitle_clips.append(subtitle_clip)

# Combinar el video original con los clips de subtítulos
final_clip = CompositeVideoClip([video_clip, *subtitle_clips])

# Añadir la pista de audio al video final
final_clip = final_clip.set_audio(audio_clip)

# Guardar el video final con subtítulos y audio
output_path = 'video_con_subtitulos_y_audio_v2.mp4'
final_clip.write_videofile(output_path, codec='libx264')



# Liberar recursos
final_clip.close()
video_clip.close()
audio_clip.close()

In [ ]:
!pip install srt

In [ ]:
import srt
from datetime import timedelta
from moviepy.editor import *

# Cargar el video original
video_clip = VideoFileClip('streetview_images3/streetview_video.mp4')

# Definir los textos de los subtítulos
subtitle_texts = [
    "Tuvo que haber un momento",
    "en el que se te aproximó la pregunta",
    "quizás con pasos más leves,", 
    "o más pesados",
    "cuando estuvo en tus manos",
    "si es que estuvo en tus manos",
    "si es que pudiste sostenerla",
    "¿Qué es una reliquia?",
    "Reliquiae: “lo que queda, los restos de la comida",
    "los sobrevivientes de una batalla, de un difunto”.",
    "La parte que se abandonó",
    "después de la pérdida o ruina del resto.",
    "Relinquere: “dejar atrás, desamparar,",
    "abandonar, rendirse”.",
    "¿Cómo se busca lo que se dejó atrás?",
    "¿Es más importante encontrarlo",
    "o llevarlo de vuelta?",
    "No pensé en tu vuelta.",
    "Sólo pensé en el viaje de ida.",
    "Dicen que te visitaron dos ángeles",
    "y te revelaron el lugar.",
    "Tres pedazos de madera y una placa.",
    "“Cualquier cosa que toma valor",
    "por su asociación con el pasado lejano”",
    '¿Qué es el pasado lejano?',
    "Tu recompensa,",
    "primera peregrina.",
    "Y aún así, dudaste.",
    "Tenés que haber dudado."
]

# Calcular la duración de cada subtítulo
total_duration = video_clip.duration
subtitle_duration = total_duration / len(subtitle_texts)

# Crear lista de subtítulos en formato SRT
subtitles = []
for i, text in enumerate(subtitle_texts):
    start_time = timedelta(seconds=i * subtitle_duration)
    end_time = timedelta(seconds=(i + 1) * subtitle_duration if i < len(subtitle_texts) - 1 else total_duration)
    subtitle = srt.Subtitle(index=i + 1, start=start_time, end=end_time, content=text)
    subtitles.append(subtitle)

# Generar el contenido del archivo SRT
srt_content = srt.compose(subtitles)

# Guardar el contenido en un archivo SRT
output_srt_path = 'subtitles.srt'
with open(output_srt_path, 'w') as f:
    f.write(srt_content)

# Liberar recursos
video_clip.close()


In [ ]:
from moviepy.editor import *

# Cargar el video original
video_clip = VideoFileClip('streetview_images3/streetview_video.mp4')

# Cargar el audio y ajustar la duración
audio_clip = AudioFileClip('streetview_images3/Arooj Aftab, Vijay Iyer, Shahzad Ismaily - Sajni (Visualizer).mp3').set_duration(video_clip.duration)

# Definir los textos de los subtítulos
subtitle_texts = [
    "Tuvo que haber un momento",
    "en el que se te aproximó la pregunta",
    "quizás con pasos más leves,", 
    "o más pesados",
    "cuando estuvo en tus manos",
    "si es que estuvo en tus manos",
    "si es que pudiste sostenerla",
    "¿Qué es una reliquia?",
    "Reliquiae: “lo que queda, los restos de la comida",
    "los sobrevivientes de una batalla, de un difunto”.",
    "La parte que se abandonó",
    "después de la pérdida o ruina del resto.",
    "Relinquere: “dejar atrás, desamparar,",
    "abandonar, rendirse”.",
    "¿Cómo se busca lo que se dejó atrás?",
    "¿Es más importante encontrarlo",
    "o llevarlo de vuelta?",
    "No pensé en tu vuelta.",
    "Sólo pensé en el viaje de ida.",
    "Dicen que te visitaron dos ángeles",
    "y te revelaron el lugar.",
    "Tres pedazos de madera y una placa.",
    "“Cualquier cosa que toma valor",
    "por su asociación con el pasado lejano”",
    '¿Qué es el pasado lejano?',
    "Tu recompensa,",
    "primera peregrina.",
    "Y aún así, dudaste.",
    "Tenés que haber dudado."
]

# Calcular la duración de cada subtítulo
total_duration = video_clip.duration
subtitle_duration = total_duration / len(subtitle_texts)

# Crear una lista de clips de subtítulos con tiempos específicos
subtitle_clips = []
for i, text in enumerate(subtitle_texts):
    start_time = i * subtitle_duration
    
    # Crear el clip de subtítulo con texto blanco
    txt_clip = TextClip(text, fontsize=26, color='white', font='Arial').set_duration(subtitle_duration).set_start(start_time)
    
    # Añadir el subtítulo al video original en la posición deseada
    video_clip = CompositeVideoClip([video_clip, txt_clip.set_position(('center', 'bottom'))])

# Añadir la pista de audio al video final
video_with_audio = video_clip.set_audio(audio_clip)

# Guardar el video final con subtítulos y audio
output_path = 'video_con_subtitulos_y_audio_v2.mp4'
video_with_audio.write_videofile(output_path, codec='libx264')

# Liberar recursos
video_with_audio.close()
video_clip.close()
audio_clip.close()


In [ ]:
# Obtener la resolución del video
width, height = final_clip.size

# Mostrar la resolución
print(f"La resolución del video es {width}x{height} píxeles.")

In [ ]:
from moviepy.editor import AudioFileClip

# Texto original dividido en subtítulos más pequeños
# Cargar el video original
video_clip = VideoFileClip('streetview_images_2/streetview_video.mp4')
audio_clip = AudioFileClip('streetview_images_2/Arooj Aftab, Vijay Iyer, Shahzad Ismaily - Sajni (Visualizer).mp3').set_duration(video_clip.duration)

subtitle_texts = [
    "Tuvo que haber un momento",
    "en el que la pregunta se te aproximó",
    "quizás con pasos más leves,", 
    "o más fuertes",
    "cuando estuvo en tus manos",
    "si es que estuvo en tus manos",
    "si es que pudiste sostenerla",
    "¿Qué es una reliquia?",
    "Reliquiae: “lo que queda, los restos de la comida",
    "los sobrevivientes de una batalla, de un difunto”.",
    "La parte que se abandonó",
    "después de la pérdida o ruina del resto.",
    "Relinquere: “dejar atrás, desamparar,",
    "abandonar, rendirse”.",
    "¿Cómo se busca lo que se dejó atrás?",
    "¿Es más importante encontrarlo",
    "o llevarlo de vuelta?",
    "No pensé en tu vuelta.",
    "Sólo pensé en el viaje de ida.",
    "Dicen que te visitaron dos ángeles",
    "y te revelaron el lugar.",
    "Tres pedazos de madera y una placa.",
    "“Cualquier cosa que toma valor",
    "por su asociación con el pasado lejano”",
    '¿Qué es el pasado lejano?',
    "Tu recompensa,",
    "primera peregrina.",
    "Y aún así, dudaste.",
    "Tenés que haber dudado."
]
# Calcular la duración de cada subtítulo
total_duration = video_clip.duration
subtitle_duration = total_duration / len(subtitle_texts)

# Crear una lista de clips de texto con tiempos específicos
subtitle_clips = []
for i, text in enumerate(subtitle_texts):
    start_time = i * subtitle_duration
    # Ajustar la posición un poco más arriba y usar color rojo oscuro
    subtitle_clip = TextClip(text, fontsize=26, color='black', font='Arial').set_position(('center', video_clip.h - 80)).set_duration(subtitle_duration).set_start(start_time)
    subtitle_clips.append(subtitle_clip)

# Combinar el video original con los clips de subtítulos
final_clip = CompositeVideoClip([video_clip, *subtitle_clips])

# Añadir la pista de audio al video final
final_clip = final_clip.set_audio(audio_clip)

# Guardar el video final con subtítulos y audio
output_path = 'video_con_subtitulos_y_audio.mp4'
final_clip.write_videofile(output_path, codec='libx264')



# Liberar recursos
final_clip.close()
video_clip.close()
audio_clip.close()

Basho

In [ ]:
import geopandas as gpd
from shapely.ops import linemerge

# Leer el archivo KMZ (se descomprime y se extrae el KML)
kmz_path1 = "D:/Fotos/CDI/maps/Basho/indicaciones/1. Fukagawa a Ataksu.kmz"

kmz_path2   = "D:/Fotos/CDI/maps/Basho/indicaciones/2. Ataksu a Fujui.kmz"

kmz_path3 = "D:/Fotos/CDI/maps/Basho/indicaciones/3. Fujui a Ogaki.kmz"

# Definir las rutas de los tres archivos KMZ
kmz_paths = [kmz_path1, kmz_path2, kmz_path3]

# Lista para almacenar las coordenadas de todos los archivos
all_coordinates = []

# Leer y procesar cada archivo KMZ
for kmz_path in kmz_paths:
    # Leer el archivo KMZ (se descomprime y se extrae el KML)
    gdf = gpd.read_file(kmz_path, driver="KML")
    
    # Asegurarse de que la geometría sea una sola línea (LineString)
    if gdf.geometry.iloc[0].geom_type == 'MultiLineString':
        line = linemerge(gdf.geometry.iloc[0])
    elif gdf.geometry.iloc[0].geom_type == 'LineString':
        line = gdf.geometry.iloc[0]
    else:
        raise ValueError('La geometría del archivo no es una LineString ni una MultiLineString.')
    
    # Proyectar a un CRS métrico (por ejemplo, UTM)
    utm_crs = gdf.estimate_utm_crs()
    line = gpd.GeoSeries([line], crs=gdf.crs).to_crs(utm_crs)[0]
    
    # Calcular la longitud total de la línea en kilómetros
    total_length_km = line.length / 1000
    print(f"Longitud total de la línea en {kmz_path}: {total_length_km} km")
    
    # Generar puntos cada 20 km
    distance_interval = 20  # 20 km
    num_points = int(total_length_km // distance_interval)
    
    points = [line.interpolate(distance_interval * i * 1000) for i in range(num_points + 1)]
    
    # Transformar los puntos de vuelta al CRS original (WGS84)
    gdf_points = gpd.GeoSeries(points, crs=utm_crs).to_crs(epsg=4326)
    
    # Crear la lista de pares de coordenadas (latitud, longitud)
    coordinates_list = [(point.y, point.x) for point in gdf_points]
    
    # Agregar las coordenadas del archivo actual a la lista global
    all_coordinates.extend(coordinates_list)

# Imprimir la lista de coordenadas concatenadas
for coord in all_coordinates:
    print(coord)



In [ ]:
import zipfile
import geopandas as gpd
from shapely.ops import linemerge
import tempfile
import os


# Leer el archivo KMZ (se descomprime y se extrae el KML)
kmz_path1 = "D:/Fotos/CDI/maps/Basho/indicaciones/1. Fukagawa a Ataksu.kmz"

kmz_path2   = "D:/Fotos/CDI/maps/Basho/indicaciones/2. Ataksu a Fujui.kmz"

kmz_path3 = "D:/Fotos/CDI/maps/Basho/indicaciones/3. Fujui a Ogaki.kmz"

# Lista para almacenar las coordenadas de todos los archivos
all_coordinates = []

# Función para extraer un KMZ a un archivo KML temporal
def extract_kmz(kmz_path):
    with zipfile.ZipFile(kmz_path, 'r') as kmz:
        # Extraer el archivo KML del KMZ
        kml_file = [f for f in kmz.namelist() if f.endswith('.kml')][0]
        # Crear un archivo temporal KML
        with tempfile.NamedTemporaryFile(delete=False, suffix='.kml') as tmp_kml:
            tmp_kml.write(kmz.read(kml_file))
            return tmp_kml.name

# Leer y procesar cada archivo KMZ
for kmz_path in kmz_paths:
    # Extraer el archivo KML temporalmente
    kml_path = extract_kmz(kmz_path)
    
    # Leer el archivo KML con GeoPandas
    gdf = gpd.read_file(kml_path, driver="KML")
    
    # Asegurarse de que la geometría sea una sola línea (LineString)
    if gdf.geometry.iloc[0].geom_type == 'MultiLineString':
        line = linemerge(gdf.geometry.iloc[0])
    elif gdf.geometry.iloc[0].geom_type == 'LineString':
        line = gdf.geometry.iloc[0]
    else:
        raise ValueError('La geometría del archivo no es una LineString ni una MultiLineString.')
    
    # Proyectar a un CRS métrico (por ejemplo, UTM)
    utm_crs = gdf.estimate_utm_crs()
    line = gpd.GeoSeries([line], crs=gdf.crs).to_crs(utm_crs)[0]
    
    # Calcular la longitud total de la línea en kilómetros
    total_length_km = line.length / 1000
    print(f"Longitud total de la línea en {kmz_path}: {total_length_km} km")
    
    # Generar puntos cada 20 km
    distance_interval = 20  # 20 km
    num_points = int(total_length_km // distance_interval)
    
    points = [line.interpolate(distance_interval * i * 1000) for i in range(num_points + 1)]
    
    # Transformar los puntos de vuelta al CRS original (WGS84)
    gdf_points = gpd.GeoSeries(points, crs=utm_crs).to_crs(epsg=4326)
    
    # Crear la lista de pares de coordenadas (latitud, longitud)
    coordinates_list = [(point.y, point.x) for point in gdf_points]
    
    # Agregar las coordenadas del archivo actual a la lista global
    all_coordinates.extend(coordinates_list)
    
    # Eliminar el archivo temporal KML
    os.remove(kml_path)

# Imprimir la lista de coordenadas concatenadas
for coord in all_coordinates:
    print(coord)


In [ ]:
all_coordinates

In [ ]:
# Definir la URL base para la API de Street View
base_url = "https://maps.googleapis.com/maps/api/streetview?size=3600x3600"

# API key
api_key = api_key

# Crear un directorio para guardar las imágenes
output_dir = 'D:/Fotos/CDI/maps/Basho/streetview'
os.makedirs(output_dir, exist_ok=True)

# Generar y mostrar URLs con coordenadas y valores aleatorios de heading y pitch
for i, coord in enumerate(all_coordinates):
    lat, lon = coord
    fov = random.randint(30, 80)
    heading = random.randint(1, 359)
    pitch = random.randint(0, 40)
    url = f"{base_url}&location={lat},{lon}&fov=80&heading={heading}&pitch={pitch}&key={api_key}"
    print(url)

    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image_path = os.path.join(output_dir, f'image_{i+1}.jpg')
        image.save(image_path)
        print(f'Imagen guardada en {image_path}')
    else:
        print(f'Error al descargar la imagen de {url}')

print('Descarga de imágenes completada.')

Antonio

In [ ]:
import zipfile
import geopandas as gpd
from shapely.ops import linemerge
import os

# Ruta del archivo KMZ
kmz_path = "D:/Fotos/CDI/maps/Antonio/Indicaciones de Heracleópolis Magna, Egipto a Monastery of Saint Anthony the great, دير القديس آلعظيم الأنبا أنطونيوس، Saint Antonius Monastery, Ras Ghareb, Egipto.kmz"

# Crear carpeta temporal para descomprimir
temp_dir = "D:/Fotos/CDI/maps/Antonio/temp_kmz"
os.makedirs(temp_dir, exist_ok=True)

# Descomprimir el archivo KMZ (es un ZIP)
with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Buscar el archivo KML dentro del KMZ
kml_file = None
for file in os.listdir(temp_dir):
    if file.endswith(".kml"):
        kml_file = os.path.join(temp_dir, file)
        break

if not kml_file:
    raise FileNotFoundError("No se encontró ningún archivo KML dentro del KMZ.")

# Leer el archivo KML con geopandas
gdf = gpd.read_file(kml_file)

# Asegurarse de que la geometría sea una sola línea (LineString)
if gdf.geometry.iloc[0].geom_type == 'MultiLineString':
    line = linemerge(gdf.geometry.iloc[0])
elif gdf.geometry.iloc[0].geom_type == 'LineString':
    line = gdf.geometry.iloc[0]
else:
    raise ValueError('La geometría no es LineString ni MultiLineString.')

# Proyectar a CRS métrico
utm_crs = gdf.estimate_utm_crs()
line_proj = gpd.GeoSeries([line], crs=gdf.crs).to_crs(utm_crs)[0]

# Calcular longitud total
total_length_km = line_proj.length / 1000
print(f"Longitud total: {total_length_km:.2f} km")

# Generar puntos cada 20 km
distance_interval = 20
num_points = int(total_length_km // distance_interval)
points = [line_proj.interpolate(i * distance_interval * 1000) for i in range(num_points + 1)]

# Transformar puntos de vuelta a WGS84
points_wgs84 = gpd.GeoSeries(points, crs=utm_crs).to_crs(epsg=4326)
coordinates = [(point.y, point.x) for point in points_wgs84]

# Mostrar coordenadas
for coord in coordinates:
    print(coord)


In [ ]:
all_coordinates = coordinates

In [ ]:
all_coordinates

In [ ]:
# Definir la URL base para la API de Street View
base_url = "https://maps.googleapis.com/maps/api/streetview?size=3600x3600"

# API key
api_key = api_key

# Crear un directorio para guardar las imágenes
output_dir = 'D:/Fotos/CDI/maps/Antonio/streetview/'
os.makedirs(output_dir, exist_ok=True)

# Generar y mostrar URLs con coordenadas y valores aleatorios de heading y pitch
for i, coord in enumerate(all_coordinates):
    lat, lon = coord
    fov = random.randint(30, 80)
    heading = random.randint(1, 359)
    pitch = random.randint(0, 40)
    url = f"{base_url}&location={lat},{lon}&fov=80&heading={heading}&pitch={pitch}&key={api_key}"
    print(url)

    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image_path = os.path.join(output_dir, f'image_{i+1}.jpg')
        image.save(image_path)
        print(f'Imagen guardada en {image_path}')
    else:
        print(f'Error al descargar la imagen de {url}')

print('Descarga de imágenes completada.')

In [ ]:
import requests
import random

def get_nearest_streetview_coords(lat, lng, api_key, radius=500):
    url = (
        f"https://maps.googleapis.com/maps/api/streetview/metadata"
        f"?location={lat},{lng}&radius={radius}&key={api_key}"
    )
    response = requests.get(url)
    data = response.json()

    if data["status"] == "OK":
        new_lat = data["location"]["lat"]
        new_lng = data["location"]["lng"]

        # Generamos valores aleatorios para la imagen
        heading = random.randint(1, 359)
        pitch = random.randint(0, 40)
        fov = random.randint(30, 80)

        return new_lat, new_lng, heading, pitch, fov
    else:
        return None



In [ ]:
def download_streetview_image(lat, lng, api_key, filename):
    base_url = "https://maps.googleapis.com/maps/api/streetview"
    params = {
        "size": "640x640",
        "location": f"{lat},{lng}",
        "key": api_key
    }
    response = requests.get(base_url, params=params)
    with open(filename, "wb") as f:
        f.write(response.content)


In [ ]:
for i, (lat, lng) in enumerate(all_coordinates):
    print(f"Procesando coordenada {i+1}: {lat}, {lng}")
    
    nueva_coord = get_nearest_streetview_coords(lat, lng, api_key, radius=10000)

    if nueva_coord:
        new_lat, new_lng = nueva_coord
        filename = f"streetview_{i+1}.jpg"
        download_streetview_image(new_lat, new_lng, api_key, filename)
        print(f"Imagen guardada en {filename} usando coordenadas ajustadas: {new_lat}, {new_lng}")
    else:
        print(f"No se encontró cobertura cercana para la coordenada {lat}, {lng}")


In [ ]:
print(all_coordinates)

In [ ]:
from shapely.geometry import Point
from math import radians, cos, sin, asin, sqrt

# Lista para guardar las coordenadas de imágenes ya utilizadas
used_coords = set()

# Para registrar coordenadas que no encontraron imagen dentro del radio máximo
no_image_coords = []

def haversine(lat1, lon1, lat2, lon2):
    # Calcula la distancia entre dos coordenadas geográficas (en km)
    R = 6371  # Radio de la Tierra
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

def buscar_imagen_mas_cercana(coord, radios=[0.5, 1, 2, 5, 10, 20]):
    lat0, lon0 = coord
    mejor_imagen = None
    menor_distancia = float("inf")

    for r in radios:
        puntos_dentro = generar_puntos_en_radio(coord, r)
        for pt in puntos_dentro:
            if pt in used_coords:
                continue
            if tiene_imagen(pt):  # tu función que verifica si hay imagen disponible
                distancia = haversine(lat0, lon0, pt[0], pt[1])
                if distancia < menor_distancia:
                    mejor_imagen = pt
                    menor_distancia = distancia

        if mejor_imagen:
            used_coords.add(mejor_imagen)
            return mejor_imagen  # salimos al encontrar la más cercana en cualquier radio

    # Si no se encontró nada dentro de 20km, devolvemos la original como "sin imagen"
    no_image_coords.append(coord)
    return coord


In [ ]:
used_coords = set()

def get_nearest_streetview_coords_multi(lat, lng, api_key, radios=[500, 1000, 2000, 5000, 10000, 20000]):
    for radius in radios:
        nearby_coord = get_nearest_streetview_coords(lat, lng, api_key, radius=radius)
        if nearby_coord and nearby_coord not in used_coords:
            used_coords.add(nearby_coord)
            return nearby_coord
    # Si no encuentra ninguna coordenada nueva en los radios, volvemos al punto original
    return (lat, lng)

for i, (lat, lng) in enumerate(all_coordinates):
    print(f"Procesando coordenada {i+1}: {lat}, {lng}")
    
    nueva_coord = get_nearest_streetview_coords_multi(lat, lng, api_key)

    new_lat, new_lng = nueva_coord
    filename = f"streetview_{i+1}.jpg"
    download_streetview_image(new_lat, new_lng, api_key, filename)
    print(f"Imagen guardada en {filename} usando coordenadas: {new_lat}, {new_lng}")



In [ ]:
def generar_puntos_en_radio(coord, radio_km, num_puntos=16):
    """
    Genera puntos alrededor de una coordenada dada en un radio (km).
    Devuelve una lista de coordenadas (lat, lon).
    """
    lat, lon = coord
    R = 6371  # Radio de la Tierra en km
    delta = radio_km / R  # distancia angular

    puntos = []
    for i in range(num_puntos):
        angle = 2 * 3.14159265 * i / num_puntos
        dlat = delta * sin(angle)
        dlon = delta * cos(angle) / cos(radians(lat))

        new_lat = lat + dlat * (180 / 3.14159265)
        new_lon = lon + dlon * (180 / 3.14159265)

        puntos.append((new_lat, new_lon))

    return puntos

In [ ]:
def tiene_imagen(coord):
    """
    Verifica si hay una imagen de Street View disponible en la coordenada dada.
    """
    lat, lon = coord
    metadata_url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={lat},{lon}&key={api_key}"

    try:
        response = requests.get(metadata_url)
        data = response.json()
        return data.get('status') == 'OK'
    except:
        return False

In [ ]:
import os
import random
import requests
from PIL import Image
from io import BytesIO
from shapely.geometry import Point
from math import radians, cos, sin, asin, sqrt

# Lista para guardar las coordenadas de imágenes ya utilizadas
used_coords = set()

# Para registrar coordenadas que no encontraron imagen dentro del radio máximo
no_image_coords = []

# Definir la URL base para la API de Street View
base_url = "https://maps.googleapis.com/maps/api/streetview?size=3600x3600"
api_key = api_key
output_dir = 'D:/Fotos/CDI/maps/Antonio/streetview/'
os.makedirs(output_dir, exist_ok=True)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

def buscar_imagen_mas_cercana(coord, index, radios=[0.5, 1, 2, 5, 10, 20]):
    lat0, lon0 = coord
    mejor_imagen = None
    menor_distancia = float("inf")

    for r in radios:
        puntos_dentro = generar_puntos_en_radio(coord, r)
        for pt in puntos_dentro:
            if pt in used_coords:
                continue
            if tiene_imagen(pt):
                distancia = haversine(lat0, lon0, pt[0], pt[1])
                if distancia < menor_distancia:
                    mejor_imagen = pt
                    menor_distancia = distancia

        if mejor_imagen:
            used_coords.add(mejor_imagen)

            # Randomizar parámetros visuales
            heading = random.randint(1, 359)
            pitch = random.randint(0, 40)

            # Construir la URL
            url = f"{base_url}&location={mejor_imagen[0]},{mejor_imagen[1]}&fov=80&heading={heading}&pitch={pitch}&key={api_key}"

            # Descargar imagen
            response = requests.get(url)
            if response.status_code == 200:
                image = Image.open(BytesIO(response.content))
                image_path = os.path.join(output_dir, f'image_{index+1}.jpg')
                image.save(image_path)
                print(f'Imagen guardada en {image_path}')
            else:
                print(f'Error al descargar la imagen de {url}')
            
            return mejor_imagen

    # Si no encuentra nada
    no_image_coords.append(coord)
    return coord


In [ ]:
for i, coord in enumerate(all_coordinates):
    buscar_imagen_mas_cercana(coord, i)

In [ ]:
coordinates2 = [(28.918443678775592, 31.635927175033654),
(28.985618542234185, 31.23408019975563),
(28.925067417010794, 32.35617972901413),
(28.921120139541337, 32.35044207078052),
(28.915477360708735, 32.34811018282845),
(28.911536715833588, 32.35487086890891),
(28.884893703139337, 32.51740494571278)]

28.915939908511344, 32.349668592374634

In [ ]:
def get_nearest_streetview_coords_multi(lat, lng, api_key, radios=[500, 1000, 2000, 5000, 10000, 20000]):
    for radius in radios:
        nearby_coord = get_nearest_streetview_coords(lat, lng, api_key, radius=radius)
        if nearby_coord and nearby_coord not in used_coords:
            used_coords.add(nearby_coord)
            return nearby_coord
    # Si no encuentra ninguna coordenada nueva en los radios, volvemos al punto original
    return (lat, lng)

for i, (lat, lng) in enumerate(coordinates2):
    print(f"Procesando coordenada {i+1}: {lat}, {lng}")
    
    nueva_coord = get_nearest_streetview_coords_multi(lat, lng, api_key)

    new_lat, new_lng = nueva_coord
    filename = f"D:/Fotos/CDI/maps/Antonio/streetview/nuevacarpeta2/streetview_{i+1}.jpg"
    download_streetview_image(new_lat, new_lng, api_key, filename)
    print(f"Imagen guardada en {filename} usando coordenadas: {new_lat}, {new_lng}")

In [ ]:
coords = get_nearest_streetview_coords({lat}, {lng}, api_key)
if coords:
    lat, lng, heading, pitch, fov = coords
    print(f"Coords: {lat}, {lng}, Heading: {heading}, Pitch: {pitch}, FOV: {fov}")

In [ ]:
def get_nearest_streetview_coords_multi(lat, lng, api_key, radios=[500, 1000, 2000, 5000, 10000, 20000]):
    for radius in radios:
        result = get_nearest_streetview_coords(lat, lng, api_key, radius=radius)
        if result:
            nearby_coord = result[:2]  # solo lat/lng para chequear en used_coords
            if nearby_coord not in used_coords:
                used_coords.add(nearby_coord)
                return result  # devuelve lat, lng, heading, pitch, fov
    # Si no encuentra ninguna coordenada nueva en los radios, devolvemos coordenadas originales + random
    fov = 90
    heading = 293.7
    pitch = 30
    return lat, lng, heading, pitch, fov


In [ ]:
import requests
from PIL import Image
from io import BytesIO

def download_streetview_image(lat, lng, api_key, filename, heading, pitch, fov):
    base_url = "https://maps.googleapis.com/maps/api/streetview"
    url = (
        f"{base_url}?size=3600x3600"
        f"&location={lat},{lng}"
        f"&heading={heading}&pitch={pitch}&fov={fov}"
        f"&key={api_key}"
    )
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(filename)
    else:
        print(f"Error al descargar imagen en {lat}, {lng} con heading {heading}")


In [ ]:
coordinates3 = [(28.9854062,31.2374789)]

In [ ]:
coordinates4 =[(28.915743,32.349324)]

In [ ]:
for i, (lat, lng) in enumerate(coordinates4):
    print(f"Procesando coordenada {i+1}: {lat}, {lng}")

    new_lat, new_lng, heading, pitch, fov = get_nearest_streetview_coords_multi(lat, lng, api_key)

    filename = f"D:/Fotos/CDI/maps/Antonio/streetview/nuevacarpeta2/streetview_9.jpg"
    download_streetview_image(new_lat, new_lng, api_key, filename, heading, pitch, fov)
    
    print(f"Imagen guardada en {filename} usando coordenadas: {new_lat}, {new_lng}")


In [ ]:
import re

def extraer_parametros_desde_url(url):
    # Expresiones regulares para extraer los valores
    lat_lng_match = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
    heading_match = re.search(r",([\d\.]+)h", url)
    pitch_match = re.search(r",([\d\.]+)t", url)
    fov_match = re.search(r",([\d\.]+)y", url)  # Opcional, no siempre se traduce directamente a fov

    if lat_lng_match:
        lat = float(lat_lng_match.group(1))
        lng = float(lat_lng_match.group(2))
    else:
        return None

    heading = float(heading_match.group(1)) if heading_match else 0
    pitch = float(pitch_match.group(1)) if pitch_match else 0
    fov = float(fov_match.group(1)) if fov_match else 90  # Por defecto 90°

    return {
        "lat": lat,
        "lng": lng,
        "heading": heading,
        "pitch": pitch,
        "fov": fov
    }


In [ ]:
import requests

def descargar_imagen_streetview_con_parametros(params, api_key, filename):
    base_url = "https://maps.googleapis.com/maps/api/streetview"
    size = "3600x3600"

    url = (
        f"{base_url}?size={size}"
        f"&location={params['lat']},{params['lng']}"
        f"&heading={params['heading']}"
        f"&pitch={params['pitch']}"
        f"&fov={params['fov']}"
        f"&key={api_key}"
    )

    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Imagen guardada en {filename}")
    else:
        print(f"No se pudo descargar la imagen. Código: {response.status_code}")

# Ejemplo de uso
url_ejemplo = "https://www.google.com.ar/maps/@28.9854062,31.2374789,3a,78.2y,319.37h,89.16t/data=!3m8!1e1!3m6!1sCIHM0ogKEICAgIDqw9j6SA!2e10!3e11!6shttps:%2F%2Flh3.googleusercontent.com%2Fgpms-cs-s%2FAIMqDu3DP-WnJ7gHz5TaGgELtOCJsKmbliH2c1wIcuCQx692YwKJQNjqxgstKBwYqPC8L20pfNEMGZAGnXnx5T5p_Mm0BSP5ZK0pnHQK7hsMYLQpIYxE1Yu8L6YUI5qt-fMDeuDu6a9g%3Dw900-h600-k-no-pi0.8421691927444783-ya230.37369928433606-ro0-fo100!7i10240!8i5120!5m1!1e4?hl=es&entry=ttu&g_ep=EgoyMDI1MDQwOS4wIKXMDSoJLDEwMjExNDU1SAFQAw%3D%3D"

params = extraer_parametros_desde_url(url_ejemplo)
if params:
    descargar_imagen_streetview_con_parametros(params, api_key, f"D:/Fotos/CDI/maps/Antonio/streetview/nuevacarpeta2/streetview_extraida.jpg")
else:
    print("No se pudieron extraer los parámetros.")


In [ ]:
url1= f""https://maps.googleapis.com/maps/api/streetview?size=600x400&location=28.9854062,31.2374789&heading=319.37&pitch=89.16&fov=78.2&key={api_key}